# Time for Search

In [20]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

## Connect to Weaviate

In [ ]:
import weaviate, os

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
        "X-Google-Api-Key": AI_STUDIO_KEY,
    }
)

client.is_ready()

In [ ]:
# import weaviate, os

# Connect with Weaviate Embedded
# client = weaviate.connect_to_embedded(
#     version="1.24.14",

#     headers={
#         "X-Google-Api-Key": AI_STUDIO_KEY,
#     })

# Connect to a Weaviate Cloud instance
# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

#     headers={
#        "X-Palm-Api-Key": AI_STUDIO_KEY,
#     }
# )

# client.is_ready()

## Vector search
[Docs - near_text](https://weaviate.io/developers/weaviate/search/similarity#an-input-medium)

In [41]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.near_text(
    query="musical instruments",
    limit=5
)

for item in response.objects:
    print(item.properties)

{'answer': 'glass', 'question': 'Flute, tumbler, shot', 'category': 'COMMON BONDS'}
{'answer': 'Tinky Winky', 'question': 'The purple Teletubby', 'category': 'RHYME TIME'}
{'answer': 'button', 'question': "Bachelor's, panic, belly", 'category': 'COMMON BONDS'}
{'answer': 'Beauty and the Beast of Burden', 'question': 'Disney musical about a pretty girl named Belle & her hideous camel', 'category': 'BEFORE & AFTER'}
{'answer': 'upside-down', 'question': 'Spanish punctuation,  pineapple cake,  people owing $400,000 on a home now worth $300,000', 'category': 'COMMON BONDS'}


## Search with filters
[Docs - Filters](https://weaviate.io/developers/weaviate/search/filters)

In [25]:
import weaviate.classes.query as wq
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.near_text(
    query="musical instruments",
    limit=5,
    filters=wq.Filter.by_property("category").like("COMMON")
)

for item in response.objects:
    print(item.properties)

{'answer': 'glass', 'question': 'Flute, tumbler, shot', 'category': 'COMMON BONDS'}
{'answer': 'button', 'question': "Bachelor's, panic, belly", 'category': 'COMMON BONDS'}
{'answer': 'upside-down', 'question': 'Spanish punctuation,  pineapple cake,  people owing $400,000 on a home now worth $300,000', 'category': 'COMMON BONDS'}
{'answer': 'Things that are spiked', 'question': 'Hair, Punch, A volleyball', 'category': 'COMMON BONDS'}
{'answer': 'things that rhyme', 'question': 'Thyme, crime, dime, grime, lime, mime, slime', 'category': 'COMMON BONDS'}


In [43]:
response = jeopardy.query.bm25(
    query="railroad",
    # query="railroad africa",
    limit=10,
)


for item in response.objects:
    print(item.properties)

{'answer': 'Trans-Siberian Railroad', 'question': 'The railroad known by this hyphenated name runs over 5,000 miles from Moscow to Vladivostok', 'category': 'TRANSPORTATION'}
{'answer': 'Amtrak', 'question': 'The corporation whose logo is seen here was created in 1971: (AKA the National Railroad Passenger Corporation)', 'category': 'BUSINESS & INDUSTRY'}
{'answer': 'San Francisco', 'question': "This city's Cable Car Museum has one from fhe first company, the Clay Street Hill Railroad", 'category': 'MUSEUMS'}
{'answer': 'a coupler', 'question': "It's a device used to connect railroad cars; today's automatic knuckle one was patented by Eli Janney in 1873", 'category': 'TRANSPORTATION'}


In [44]:
response = jeopardy.query.bm25(
    query="railroad africa",
    query_properties=["answer"],
    limit=10,
)


for item in response.objects:
    print(item.properties)

{'answer': 'Africa', 'question': 'The black-footed penguin is native to the south of this continent, whose north had the highest temperature on record', 'category': 'SCIENCE & NATURE'}
{'answer': 'Trans-Siberian Railroad', 'question': 'The railroad known by this hyphenated name runs over 5,000 miles from Moscow to Vladivostok', 'category': 'TRANSPORTATION'}


## Hybrid search
[Docs - hybrid](https://weaviate.io/developers/weaviate/search/hybrid)

In [50]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="rail, trains and cars",
    alpha=0.8,
    limit=5,
)

for item in response.objects:
    print(item.properties["answer"], " || ", item.properties["question"])

a coupler  ||  It's a device used to connect railroad cars; today's automatic knuckle one was patented by Eli Janney in 1873
Trans-Siberian Railroad  ||  The railroad known by this hyphenated name runs over 5,000 miles from Moscow to Vladivostok
San Francisco  ||  This city's Cable Car Museum has one from fhe first company, the Clay Street Hill Railroad
Surrender, Dorothy Hamill!  ||  The Wicked Witch of the West's skywritten demand of the 1976 ladies' Olympic figure skating champion
Amtrak  ||  The corporation whose logo is seen here was created in 1971: (AKA the National Railroad Passenger Corporation)


In [63]:
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="train railroad cars",
    query_properties=["answer^5", "question"],
    alpha=0.5,
    limit=5,
)

for item in response.objects:
    print(item.properties["answer"], " || ", item.properties["question"])

Trans-Siberian Railroad  ||  The railroad known by this hyphenated name runs over 5,000 miles from Moscow to Vladivostok
a coupler  ||  It's a device used to connect railroad cars; today's automatic knuckle one was patented by Eli Janney in 1873
San Francisco  ||  This city's Cable Car Museum has one from fhe first company, the Clay Street Hill Railroad
Amtrak  ||  The corporation whose logo is seen here was created in 1971: (AKA the National Railroad Passenger Corporation)
pasta  ||  Type of food that comes in shapes of bow ties, elbows & wagon wheels


In [76]:
from weaviate.classes.query import MetadataQuery
jeopardy = client.collections.get("Jeopardy")

response = jeopardy.query.hybrid(
    query="train railroad cars",
    query_properties=["answer^5", "question"],
    alpha=0.5,
    return_metadata=MetadataQuery(explain_score=True),
    limit=5,
)

for item in response.objects:
    print(item.metadata.explain_score)
    print(item.properties["answer"], " || ", item.properties["question"])


Hybrid (Result Set keyword,bm25) Document 9fbdb4aa-9724-4242-9f5e-b4604019c6c3: original score 2.207006, normalized score: 0.5 - 
Hybrid (Result Set vector,hybridVector) Document 9fbdb4aa-9724-4242-9f5e-b4604019c6c3: original score 0.585617, normalized score: 0.48361167
Trans-Siberian Railroad  ||  The railroad known by this hyphenated name runs over 5,000 miles from Moscow to Vladivostok

Hybrid (Result Set keyword,bm25) Document 558f18af-870d-451f-a03e-ef46fe2bde0a: original score 2.049873, normalized score: 0.43684438 - 
Hybrid (Result Set vector,hybridVector) Document 558f18af-870d-451f-a03e-ef46fe2bde0a: original score 0.5919533, normalized score: 0.5
a coupler  ||  It's a device used to connect railroad cars; today's automatic knuckle one was patented by Eli Janney in 1873

Hybrid (Result Set keyword,bm25) Document a047cdc0-9a37-454e-9ab7-400319d1ec0c: original score 0.96299255, normalized score: 0 - 
Hybrid (Result Set vector,hybridVector) Document a047cdc0-9a37-454e-9ab7-40031

# ————————— STOP HERE —————————

## Retrieval Augmented Generation!!!
[Docs - RAG](https://weaviate.io/developers/weaviate/search/generative)

In [ ]:
questions = client.collections.get("Questions")

response = questions.generate.near_text(
    query="musical instruments",
    limit=4,
    single_prompt="Write a short tweet about: {question}"
)

for item in response.objects:
    print(item.properties)
    print(item.generated)


In [ ]:
questions = client.collections.get("Questions")

response = questions.generate.near_text(
    query="musical instruments",
    limit=4,
    grouped_task="Explain what this content is about."
)

print (response.generated)